In [1]:
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, LSTM

class MusicSentencesDataGenerator(keras.utils.Sequence):

    '''
    song_IDs : np.ndarray
        List of song IDs in the data
    labels : dict
        Dictionary mapping each song ID to time period label
    data_path : string
        Path of the .npy data files
    batch_size : int
        Batch size
    sentence_length : int (default 30)
        Number of seconds each data sample (.npy file) represents
    word_length : int (default 4)
        Number of seconds each word represents; each sentence is split into words of this length
    SR : int (default 44100)
        Sample rate; number of values per second
    '''
    def __init__(self, song_IDs, labels, data_path, batch_size, sentence_length=30,
                 word_length=5, SR=44100, shuffle=True):
        self.song_IDs = song_IDs
        self.labels = labels
        self.data_path = data_path
        self.batch_size = batch_size
        self.sentence_length = sentence_length
        self.word_length = word_length
        self.SR = SR
        self.data_dim = (sentence_length // word_length, word_length * SR)
        self.shuffle = shuffle
        self.labels2ind = list(set(labels.values()))
        self.on_epoch_end()
        
    def __len__(self):
        'Returns the number of batches per epoch'
        return int(np.floor(len(self.song_IDs) / self.batch_size))
    
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        song_IDs_temp = [self.song_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(song_IDs_temp)

        return X, y
    
    def __data_generation(self, song_IDs_temp):
        'Generates data containing batch_size samples' # X : (batch_size, num_words, word_length)
        # Initialization
        X = np.empty((self.batch_size, *self.data_dim))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(song_IDs_temp):
            # Store sample
            X[i,] = (np.load(self.data_path + str(ID) + '.npy')).reshape((self.data_dim))

            # Store class
            y[i] = self.labels[ID]
        
        # Categorical to numerical labels
        y = [self.labels2ind.index(a) for a in y]

        return X, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.song_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.song_IDs)

Using TensorFlow backend.


In [2]:
# Generate npy files to test
for i in range(100):
    data = np.random.rand(10 * 30)
    np.save("sample_data/" + str(i), data)

In [5]:
# Testing DataGenerator
ids =np.arange(100)

shuffled = np.random.permutation(ids)
train_ids = shuffled[:int(len(shuffled) * 0.8)]
test_ids = shuffled[int(len(shuffled) * 0.8):]

labels = {k: k%3 for k in ids}

train_generator = MusicSentencesDataGenerator(train_ids, labels, "sample_data/", batch_size=20, SR=10)
test_generator = MusicSentencesDataGenerator(test_ids, labels, "sample_data/", batch_size=20, SR=10)

model = Sequential()
model.add(LSTM(100, input_shape=(6, 5*10), return_sequences=True))
model.add(Flatten())
model.add(Dense(3, activation='sigmoid'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

EPOCHS = 100
BATCH_SIZE=20
model.fit_generator(generator=train_generator, validation_data=test_generator, epochs=EPOCHS)

scores = model.evaluate_generator(test_generator, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/100
4/4 [==============================] - 1s 154ms/step - loss: 1.1143 - accuracy: 0.2875 - val_loss: 1.0960 - val_accuracy: 0.4000
Epoch 2/100
4/4 [==============================] - 0s 11ms/step - loss: 1.0999 - accuracy: 0.3500 - val_loss: 1.1223 - val_accuracy: 0.3000
Epoch 3/100
4/4 [==============================] - 0s 29ms/step - loss: 1.0864 - accuracy: 0.3500 - val_loss: 1.1122 - val_accuracy: 0.2500
Epoch 4/100
4/4 [==============================] - 0s 28ms/step - loss: 1.0739 - accuracy: 0.4875 - val_loss: 1.1071 - val_accuracy: 0.4000
Epoch 5/100
4/4 [==============================] - 0s 20ms/step - loss: 1.0662 - accuracy: 0.5000 - val_loss: 1.0983 - val_accuracy: 0.3500
Epoch 6/100
4/4 [==============================] - 0s 26ms/step - loss: 1.0577 - accuracy: 0.5125 - val_loss: 1.0976 - val_accuracy: 0.3500
Epoch 7/100
4/4 [==============================] - 0s 23ms/step - loss: 1.0519 - accuracy: 0.6750 - val_loss: 1.0853 - val_accuracy: 0.5000
Epoch 8/100
4/4 [==

4/4 [==============================] - 0s 34ms/step - loss: 0.0153 - accuracy: 1.0000 - val_loss: 3.1121 - val_accuracy: 0.4000
Epoch 60/100
4/4 [==============================] - 0s 25ms/step - loss: 0.0139 - accuracy: 1.0000 - val_loss: 3.1680 - val_accuracy: 0.4000
Epoch 61/100
4/4 [==============================] - 0s 26ms/step - loss: 0.0135 - accuracy: 1.0000 - val_loss: 3.1966 - val_accuracy: 0.4000
Epoch 62/100
4/4 [==============================] - 0s 20ms/step - loss: 0.0124 - accuracy: 1.0000 - val_loss: 3.2193 - val_accuracy: 0.4000
Epoch 63/100
4/4 [==============================] - 0s 26ms/step - loss: 0.0116 - accuracy: 1.0000 - val_loss: 3.2491 - val_accuracy: 0.4000
Epoch 64/100
4/4 [==============================] - 0s 27ms/step - loss: 0.0115 - accuracy: 1.0000 - val_loss: 3.3040 - val_accuracy: 0.4000
Epoch 65/100
4/4 [==============================] - 0s 26ms/step - loss: 0.0103 - accuracy: 1.0000 - val_loss: 3.3063 - val_accuracy: 0.4000
Epoch 66/100
4/4 [========